In [4]:
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
!pip install -qqq bitsandbytes
!pip install -qqq torch
!pip install  -qqq -U git+https://github.com/huggingface/transformers
!pip install -qqq -U git+https://github.com/huggingface/peft
!pip install -qqq -U git+https://github.com/huggingface/accelerate
!pip install -qqq datasets
!pip install -qqq loralib
!pip install -qqq einops

In [7]:
PEFT_MODEL = "fine-tuned-models/minichat"

config = PeftConfig.from_pretrained(PEFT_MODEL)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

In [8]:
# Load sentence transformer model
from sentence_transformers import SentenceTransformer
filter_model = SentenceTransformer('all-MiniLM-L6-v2')

In [9]:
# Get vector from an input text by Sentence Transformer
def get_vector(text):
    text_vector = filter_model.encode(text)
    return torch.tensor(text_vector)

In [10]:
# Vectorization of "movie recommendation"
valid_query = "movie recommendation"
valid_vector = get_vector(valid_query)
valid_vector.shape

torch.Size([384])

In [11]:
# Generate response based on a suitable prompt
def bot_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)

    # Generate answer
    output_sequences = model.generate(
        **inputs,
        max_length=1000,
        num_return_sequences=1,
        temperature=0.1,  # Adjust for creativity
        top_p=0.1,  # Nucleus sampling
        top_k=50,  # Top-k sampling
        no_repeat_ngram_size=2  # Prevent repeating n-grams
    )
    
    # Process the output to remove the input question from the response if it gets included
    answer = tokenizer.decode(output_sequences[0], skip_special_tokens=True).replace(prompt, "").strip()
    return answer

In [12]:
# Connect to the Qdrant database
from qdrant_client import models, QdrantClient
from qdrant_client.http.models import CollectionDescription, VectorParams

client = QdrantClient(host='localhost', port=6333)

In [13]:
# Perform vector database search
def database_search(input_text):
    results = []
    hits = client.search(
        collection_name='movies',
        query_vector=filter_model.encode(input_text).tolist(),
        limit = 2
    )
    for idx, hit in enumerate(hits):
        result = {}
        result['original_title'] = hit.payload['original_title']
        result['genres'] = hit.payload['genres']
        result['overview'] = hit.payload['overview']
        results.append(result)
    return results

In [14]:
def answer_with_query(text, results):
    # Constructing a detailed prompt that structures the results as individual pieces of evidence
    evidence_list = "\n".join([f"Title: {result['original_title']}\nGenres: {result['genres']}\nOverview: {result['overview']}" for result in results])
    # prompt = f"Based on the following information, answer the question '{text}':\n\n{evidence_list}\n\nAnswer:"
    
    # answer = bot_response(prompt)
    return evidence_list

In [15]:
intentions = [
    "Can you recommend to me a movie similar to Harry Potter?",
    "What's the weather like today?",
    "Book a table for two at a Mexican restaurant tonight.",
    "Tell me a joke.",
    "How can I reset my password?",
    "Play some music from the 90s.",
    "What was the highest-grossing film in 2020?",
    "Find me a flight to New York next weekend.",
    "I need a recipe for a chocolate cake.",
    "What's the latest news on technology?",
    "Show me my photos from Paris.",
    "How do I make a Manhattan cocktail?",
    "Suggest a good workout for abs.",
    "What are some vegan restaurants nearby?",
    "Is there a good comedy movie to watch?",
    "I want to learn Python programming.",
    "Where can I watch The Office?",
    "Give me directions to the nearest gas station.",
    "What are the symptoms of COVID-19?",
    "Tell me about the life of Albert Einstein.",
    "Show me the latest horror films.",
    "What animated movies are recommended for kids?",
    "List all movies starring Tom Hanks.",
    "Who directed Pulp Fiction?",
    "Find movies similar to La La Land.",
    "What's the plot of The Matrix?",
    "Give me the soundtrack list from Guardians of the Galaxy.",
    "How many Oscars did Forrest Gump win?",
    "Is 'Joker' suitable for teenagers?",
    "What are some must-watch classic films?",
    "Play the trailer for 'Avengers: Infinity War'.",
    "I'm in the mood for a comedy. Any suggestions?",
    "What films are based on Stephen King's books?",
    "When was Schindler's List released?",
    "Find me a movie that will make me cry.",
    "What are the best sci-fi movies of the last decade?",
    "Which movies are in the Marvel Cinematic Universe?",
    "Tell me about the directorial style of Wes Anderson.",
    "How do I access movies on Netflix?",
    "Rate 'Mad Max: Fury Road' out of 10."
]

In [16]:
from torch.nn.functional import cosine_similarity

for intention in intentions:
    intention_vector = get_vector(intention)
    similarity = cosine_similarity(intention_vector.unsqueeze(0), valid_vector.unsqueeze(0))
    print(f"Question: {intention}")
    if similarity < 0.5:
        prompt = f"Answer the following question:\n{intention}"
        print(bot_response(prompt))
    if similarity >= 0.5:
        prompt = f"For the information mentioned, describe its genres and any notable themes:\n{intention}"
        bot_answer = bot_response(prompt)
        print(f"First answer:\n{bot_answer}")
        results = database_search(bot_answer)
        evidence_list = answer_with_query(intention, results)
        print(f"Databse search results:\n{evidence_list}")
        # print(f"Final answer:\n{answer}")
    print("============================================================")
        

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Question: Can you recommend to me a movie similar to Harry Potter?


/home/nhannguyen/.local/share/virtualenvs/movie-recommendation-LLM-HHW9OGcc/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/nhannguyen/.local/share/virtualenvs/movie-recommendation-LLM-HHW9OGcc/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


First answer:
1. The film is directed by Peter Jackson, who directed the Lord of the Rings film trilogy, and stars Ian McKellen as Gandalf, Ian Holm as Elrond, Billy Boyd as Frodo Baggins, Dominic Monaghan as Sam, John Rhys-Davies as Gollum, Orlando Bloom as Legolas, Sean Astin as Bard, Hugo Weaving as Thorin Oakenshield, Christopher Lee as Saruman, Andy Serkis as the Goblin King, Liv Tyler as Arwen, Cate Blanchett as Galadriel, Richard Armitage as Aragorn, Viggo Mortensen as The Nazgul, Peter Capaldi as Mr. Ollivander, Sophie Hunter as Eowyn, Mark Strong as Merry, Alan Lee, James Cosmo, David Bradley, Tom Bower, Joe Absolom, Rob Kazinsky, Simon Rhee, Daniel Mays, Michael Sheen, Stephen Fry and many more actors.
2. It is a fantasy film that takes place in Middle-earth, a fictional world created by J.R. R. Tolkien, where the story is set in the year 3019, after the War of The Last Alliance, when the Dark Lord Sauron is defeated and the Ring of Power is destroyed. Gandelf, Elron, Froddo,